<a href="https://colab.research.google.com/github/avi26-git/ML/blob/master/Credit%20Card%20Approval%20Predictor/Credit%20Card%20Approval%20Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automatic Credit Card Approval Prediction**

---



Credit card applications get rejected many times because of various reasons. e.g., high loan balances, low income levels or fault payments.
This can be a mundane, time taking and error prone task for humans, here we'll use machine learning to automate this process.
The dataset that would be used is : [Credit Card Approval dataset](http://archive.ics.uci.edu/ml/datasets/credit+approval) from the UCI Machine Learning Repository.

The structure of this notebook would be:


1.   Loading and viewing the dataset.
2.   Data preprocessing.
3.   EDA.
4.   ML Model.



## 1. Loading and viewing data.



### Importing necessary libraries

In [9]:
#Importing necessary libraries
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

# Load dataset
cc_applications = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data", header=None)

#printing first 5 lines
print(cc_applications.head())

  0      1   2  3  4  5  6   7  8  9   10 11 12     13   14 15
0  b  30.83 0.0  u  g  w  v 1.2  t  t   1  f  g  00202    0  +
1  a  58.67 4.5  u  g  q  h 3.0  t  t   6  f  g  00043  560  +
2  a  24.50 0.5  u  g  q  h 1.5  t  f   0  f  g  00280  824  +
3  b  27.83 1.5  u  g  w  v 3.8  t  t   5  t  g  00100    3  +
4  b  20.17 5.6  u  g  w  v 1.7  t  f   0  f  s  00120    0  +


**Exploratory Analysis and Data Transformations**

A review of the [codebook](http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.names) shows that all of the values in the dataset have been converted to meaningless symbols to protect the confidentiality of the data.

The probable features in a typical credit card application are Gender, Age, Debt, Married, BankCustomer, EducationLevel, Ethnicity, YearsEmployed, PriorDefault, Employed, CreditScore, DriversLicense, Citizen, ZipCode, Income and finally the ApprovalStatus.

In [10]:
# Print summary statistics
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print("\n")

# Print DataFrame information
cc_apps_info = cc_apps.info()
print(cc_apps_info)

print("\n")

# Inspect missing values in the dataset
# ... YOUR CODE FOR TASK 2 ...
cc_apps.tail(17)

         2     7     10       14
count 690.0 690.0 690.0    690.0
mean    4.8   2.2   2.4   1017.4
std     5.0   3.3   4.9   5210.1
min     0.0   0.0   0.0      0.0
25%     1.0   0.2   0.0      0.0
50%     2.8   1.0   0.0      5.0
75%     7.2   2.6   3.0    395.5
max    28.0  28.5  67.0 100000.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null object
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null int64
15    690 non-null object
dtypes: float64(2), int64(2), object(12)
memory usage: 86.3+ KB
None




,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
673,?,29.50,2.0,y,p,e,h,2.0,f,f,0,f,g,00256,17,-
674,a,37.33,2.5,u,g,i,h,0.2,f,f,0,f,g,00260,246,-
675,a,41.58,1.0,u,g,aa,v,0.7,f,f,0,f,g,00240,237,-
676,a,30.58,10.7,u,g,q,h,0.1,f,t,12,t,g,00129,3,-
677,b,19.42,7.2,u,g,m,v,0.0,f,t,1,f,g,00100,1,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.1,y,p,e,h,1.2,f,f,0,f,g,00260,0,-
686,a,22.67,0.8,u,g,c,v,2.0,f,t,2,t,g,00200,394,-
687,a,25.25,13.5,y,p,ff,ff,2.0,f,t,1,t,g,00200,1,-
688,b,17.92,0.2,u,g,aa,v,0.0,f,f,0,f,g,00280,750,-
